In [4]:
from argparse import ArgumentParser
import mmcv
import numpy as np
import os
import os.path as osp
import numpy as np
from shapely.geometry import Polygon, LineString, MultiLineString, GeometryCollection
from shapely.algorithms.polylabel import polylabel
import math
import cv2
import json
import pickle

def load_ann(filename):
    with open(filename) as f:
        texts = f.readlines()
    num_objs = len(texts)
    polygon_list = []
    for i in range(num_objs):
        try:
            box = texts[i].split(',')
            box = np.array(box, dtype=np.float32)
            box[4::2] += box[0]
            box[5::2] += box[1]
            box = box[4:]
            hard = 0
            polygon = Polygon(box.reshape((-1, 2)))
        except Exception as err:
            print(filename, texts[i], err)
        polygon_list.append([polygon, hard])
    return polygon_list

def load_pred(filename):
    with open(filename, 'rb') as f:
        array = pickle.load(f)
    pred_polygon_list = []
    for ix, pred_per_sample in enumerate(array):
        polygon_list = []
        for pred_one_out in pred_per_sample:
            coords = pred_one_out[:-1]
            score = float(pred_one_out[-1])
#             coords = pred_one_out
#             score = 1
            if score < 0.95:
                continue
            polygon = Polygon(coords.reshape((-1, 2)))
#             if not polygon.is_valid:
#                 print(ix, polygon)
            polygon_list.append(polygon)
        pred_polygon_list.append(polygon_list)
    return pred_polygon_list

def polygon_iou(poly1, poly2):
    """
    Intersection over union between two shapely polygons.
    """
    iou = 0.0
    if poly1.intersects(poly2):
        try:
            inter_area = poly1.intersection(poly2).area
            union_area = poly1.area + poly2.area - inter_area
            iou = float(inter_area) / union_area
        except Exception as err:
            iou = 0.5
#             print(poly1)
    return iou

def evaluation(pred_polygon_list, gt_polygon_list):
    assert len(pred_polygon_list)==len(gt_polygon_list), "sample numbers should be same"
    num_gt = 0
    num_pred = 0
    num_gt_correct = 0
    num_pred_correct = 0
    for sample_idx in range(len(pred_polygon_list)):
        gt_polygons = gt_polygon_list[sample_idx]
        pred_polygons = pred_polygon_list[sample_idx]
        num_gt += len(gt_polygons)
        num_pred += len(pred_polygons)
        for gt_polygon, hard in gt_polygons:
            num_gt -= 1 if hard else 0
            gt_correct = False
            for pred_polygon in pred_polygons:
                iou = polygon_iou(gt_polygon, pred_polygon)
                if iou >= 0.5:
                    num_pred_correct += 1 if not hard else 0
                    num_pred -= 1 if hard else 0
                    gt_correct = True
            num_gt_correct += 1 if (gt_correct and not hard) else 0
    p = 1.0 * num_pred_correct / max(num_pred, 1e-10)
    r = 1.0 * num_gt_correct / max(num_gt, 1e-10)
    f = 2.0 * p * r / max(p + r, 1e-10)
    print("num_pred correct/all (%d/%d); num_gt correct/all (%d/%d)"%(num_pred_correct, num_pred, num_gt_correct, num_gt))
    print("precision: %f%%"%(p*100.0))
    print("recall: %f%%"%(r*100.0))
    print("f-measure: %f%%"%(f*100.0))
        
# load pred pickle
pickle_file = '../../experiments/CTW_cheby/CTW_cheby_08071114.pkl'
pred_polygon_list = load_pred(pickle_file)
# load gt ann
img_prefix = '/home/wff/curve-text/data/CTW1500/'
test_file = os.path.join(img_prefix, 'ImageSets/Main/test.txt')
img_ids = mmcv.list_from_file(test_file)
gt_polygon_list = []
for img_id in img_ids:
    ann_file = os.path.join(img_prefix, 'Annotations', img_id + '.txt')
    gt_polygon_list.append(load_ann(ann_file))
# evaluation
result = evaluation(pred_polygon_list, gt_polygon_list)

num_pred correct/all (2444/2921); num_gt correct/all (2444/3068)
precision: 83.669976%
recall: 79.661017%
f-measure: 81.616297%
